### Neural Network OOP code
1.  Object-Oriented Structure:
2.  A NeuralNetwork Model class handles model training and prediction.
3.  It can be easily reused, extended, or swapped for another model.

A prediction is made using new data.



In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

class NeuralNetworkModel:
    def __init__(self, hidden_layer_sizes=(10,), max_iter=1000, random_state=42):
        """Initializes the neural network model."""
        self.model = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes,
                                   max_iter=max_iter,
                                   random_state=random_state)

    def train(self, X_train, y_train):
        """Trains the neural network with training data."""
        self.model.fit(X_train, y_train)

    def predict(self, X_test):
        """Makes predictions on new data."""
        return self.model.predict(X_test)

    def evaluate(self, X_test, y_test):
        """Evaluates the model's performance."""
        predictions = self.predict(X_test)
        return accuracy_score(y_test, predictions)


# Example usage with Iris dataset
if __name__ == "__main__":
    # Load sample data
    iris = load_iris()
    X, y = iris.data, iris.target

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the model
    nn = NeuralNetworkModel(hidden_layer_sizes=(5, 5))
    nn.train(X_train, y_train)

    # Evaluate and make a prediction
    accuracy = nn.evaluate(X_test, y_test)
    print(f"Model accuracy: {accuracy:.2f}")

    sample = X_test[0].reshape(1, -1)
    prediction = nn.predict(sample)
    print(f"Prediction for sample {sample}: {iris.target_names[prediction[0]]}")


Model accuracy: 0.90
Prediction for sample [[6.1 2.8 4.7 1.2]]: virginica


/home/codespace/.local/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


### Second Example

In [5]:
import numpy as np

# Activation functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        # Initialize weights and biases
        self.learning_rate = learning_rate
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.bias_hidden = np.zeros((1, hidden_size))
        
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_output = np.zeros((1, output_size))
    
    def forward(self, X):
        # Feedforward step
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.final_output = sigmoid(self.final_input)
        return self.final_output

    def backward(self, X, y, output):
        # Calculate error
        error = y - output
        
        # Calculate gradients
        d_output = error * sigmoid_derivative(output)
        error_hidden = d_output.dot(self.weights_hidden_output.T)
        d_hidden = error_hidden * sigmoid_derivative(self.hidden_output)
        
        # Update weights and biases
        self.weights_hidden_output += self.hidden_output.T.dot(d_output) * self.learning_rate
        self.bias_output += np.sum(d_output, axis=0, keepdims=True) * self.learning_rate
        
        self.weights_input_hidden += X.T.dot(d_hidden) * self.learning_rate
        self.bias_hidden += np.sum(d_hidden, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs=10000):
        for i in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output)
            if i % 1000 == 0:
                loss = np.mean((y - output) ** 2)
                print(f"Epoch {i}, Loss: {loss:.4f}")
    
    def predict(self, X):
        output = self.forward(X)
        return np.round(output)

# Example: XOR problem
if __name__ == "__main__":
    # Input data (XOR)
    X = np.array([[0, 0],
                  [0, 1],
                  [1, 0],
                  [1, 1]])
    
    # Labels
    y = np.array([[0],
                  [1],
                  [1],
                  [0]])
    
    # Create and train the model
    model = SimpleNeuralNetwork(input_size=2, hidden_size=4, output_size=1)
    model.train(X, y, epochs=10000)

    # Predict
    predictions = model.predict(X)
    print("Predictions:")
    print(predictions)


Epoch 0, Loss: 0.3273
Epoch 1000, Loss: 0.1603
Epoch 2000, Loss: 0.0489
Epoch 3000, Loss: 0.0187
Epoch 4000, Loss: 0.0104
Epoch 5000, Loss: 0.0070
Epoch 6000, Loss: 0.0051
Epoch 7000, Loss: 0.0041
Epoch 8000, Loss: 0.0033
Epoch 9000, Loss: 0.0028
Predictions:
[[0.]
 [1.]
 [1.]
 [0.]]


## Third example

#### A NeuralNetworkModel class handles model training and prediction.

#### It can be easily reused, extended, or swapped for another model.

#### A prediction is made using new data.

In [3]:
import numpy as np

class Layer:
    def __init__(self, input_size, output_size):
        # Initialize weights and biases
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.zeros((1, output_size))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        return self.activation(self.z)
    
    def activation(self, z):
        # Sigmoid activation function
        return 1 / (1 + np.exp(-z))
    
    def backward(self, output_gradient, learning_rate):
        # Calculate gradients
        activation_gradient = self.z * (1 - self.z)  # Derivative of sigmoid
        weighted_gradient = output_gradient * activation_gradient
        self.weights_gradient = np.dot(self.inputs.T, weighted_gradient)
        self.biases_gradient = np.sum(weighted_gradient, axis=0, keepdims=True)
        
        # Update weights and biases
        self.weights -= learning_rate * self.weights_gradient
        self.biases -= learning_rate * self.biases_gradient
        
        # Return the gradient for the previous layer
        return np.dot(weighted_gradient, self.weights.T)


class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        
    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs
    
    def backward(self, output_gradient, learning_rate):
        for layer in reversed(self.layers):
            output_gradient = layer.backward(output_gradient, learning_rate)
    
    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            # Forward pass
            output = self.forward(x_train)
            # Compute loss (Mean Squared Error)
            loss = np.mean((output - y_train) ** 2)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")
            # Backward pass
            output_gradient = 2 * (output - y_train) / y_train.size
            self.backward(output_gradient, learning_rate)


# Example usage
if __name__ == "__main__":
    # Sample data (XOR problem)
    x_train = np.array([[0, 0],
                        [0, 1],
                        [1, 0],
                        [1, 1]])
    
    y_train = np.array([[0], [1], [1], [0]])  # XOR output
    
    # Create a simple neural network with one hidden layer
    layers = [
        Layer(input_size=2, output_size=2),  # Hidden layer
        Layer(input_size=2, output_size=1)   # Output layer
    ]
    
    nn = NeuralNetwork(layers)
    nn.train(x_train, y_train, epochs=10000, learning_rate=0.1)

Epoch 1/10000, Loss: 0.2500021268935867
Epoch 2/10000, Loss: 0.2500021287683957
Epoch 3/10000, Loss: 0.2500021306456893
Epoch 4/10000, Loss: 0.2500021325254717
Epoch 5/10000, Loss: 0.2500021344077475
Epoch 6/10000, Loss: 0.25000213629252094
Epoch 7/10000, Loss: 0.2500021381797966
Epoch 8/10000, Loss: 0.2500021400695789
Epoch 9/10000, Loss: 0.2500021419618721
Epoch 10/10000, Loss: 0.25000214385668085
Epoch 11/10000, Loss: 0.2500021457540096
Epoch 12/10000, Loss: 0.2500021476538627
Epoch 13/10000, Loss: 0.25000214955624483
Epoch 14/10000, Loss: 0.2500021514611602
Epoch 15/10000, Loss: 0.2500021533686137
Epoch 16/10000, Loss: 0.2500021552786096
Epoch 17/10000, Loss: 0.2500021571911524
Epoch 18/10000, Loss: 0.2500021591062468
Epoch 19/10000, Loss: 0.2500021610238971
Epoch 20/10000, Loss: 0.25000216294410815
Epoch 21/10000, Loss: 0.2500021648668843
Epoch 22/10000, Loss: 0.25000216679223025
Epoch 23/10000, Loss: 0.2500021687201505
Epoch 24/10000, Loss: 0.25000217065064967
Epoch 25/10000, Los

/tmp/ipykernel_2672/513021690.py:16: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
/tmp/ipykernel_2672/513021690.py:20: RuntimeWarning: overflow encountered in multiply
  activation_gradient = self.z * (1 - self.z)  # Derivative of sigmoid
/tmp/ipykernel_2672/513021690.py:21: RuntimeWarning: invalid value encountered in multiply
  weighted_gradient = output_gradient * activation_gradient


Epoch 3292/10000, Loss: nan
Epoch 3293/10000, Loss: nan
Epoch 3294/10000, Loss: nan
Epoch 3295/10000, Loss: nan
Epoch 3296/10000, Loss: nan
Epoch 3297/10000, Loss: nan
Epoch 3298/10000, Loss: nan
Epoch 3299/10000, Loss: nan
Epoch 3300/10000, Loss: nan
Epoch 3301/10000, Loss: nan
Epoch 3302/10000, Loss: nan
Epoch 3303/10000, Loss: nan
Epoch 3304/10000, Loss: nan
Epoch 3305/10000, Loss: nan
Epoch 3306/10000, Loss: nan
Epoch 3307/10000, Loss: nan
Epoch 3308/10000, Loss: nan
Epoch 3309/10000, Loss: nan
Epoch 3310/10000, Loss: nan
Epoch 3311/10000, Loss: nan
Epoch 3312/10000, Loss: nan
Epoch 3313/10000, Loss: nan
Epoch 3314/10000, Loss: nan
Epoch 3315/10000, Loss: nan
Epoch 3316/10000, Loss: nan
Epoch 3317/10000, Loss: nan
Epoch 3318/10000, Loss: nan
Epoch 3319/10000, Loss: nan
Epoch 3320/10000, Loss: nan
Epoch 3321/10000, Loss: nan
Epoch 3322/10000, Loss: nan
Epoch 3323/10000, Loss: nan
Epoch 3324/10000, Loss: nan
Epoch 3325/10000, Loss: nan
Epoch 3326/10000, Loss: nan
Epoch 3327/10000, Lo

### Fourth Example

In [2]:
import numpy as np

class Layer:
    def __init__(self, input_size, output_size):
        # Initialize weights and biases
        self.weights = np.random.randn(input_size, output_size) * 0.01
        self.biases = np.zeros((1, output_size))
        
    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(inputs, self.weights) + self.biases
        return self.activation(self.z)
    
    def activation(self, z):
        # Sigmoid activation function
        return 1 / (1 + np.exp(-z))
    
    def backward(self, output_gradient, learning_rate):
        # Calculate gradients
        activation_gradient = self.z * (1 - self.z)  # Derivative of sigmoid
        weighted_gradient = output_gradient * activation_gradient
        self.weights_gradient = np.dot(self.inputs.T, weighted_gradient)
        self.biases_gradient = np.sum(weighted_gradient, axis=0, keepdims=True)
        
        # Update weights and biases
        self.weights -= learning_rate * self.weights_gradient
        self.biases -= learning_rate * self.biases_gradient
        
        # Return the gradient for the previous layer
        return np.dot(weighted_gradient, self.weights.T)


class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers
        
    def forward(self, inputs):
        for layer in self.layers:
            inputs = layer.forward(inputs)
        return inputs
    
    def backward(self, output_gradient, learning_rate):
        for layer in reversed(self.layers):
            output_gradient = layer.backward(output_gradient, learning_rate)
    
    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            # Forward pass
            output = self.forward(x_train)
            # Compute loss (Mean Squared Error)
            loss = np.mean((output - y_train) ** 2)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss}")
            # Backward pass
            output_gradient = 2 * (output - y_train) / y_train.size
            self.backward(output_gradient, learning_rate)


# Example usage
if __name__ == "__main__":
    # Sample data (XOR problem)
    x_train = np.array([[0, 0],
                        [0, 1],
                        [1, 0],
                        [1, 1]])
    
    y_train = np.array([[0], [1], [1], [0]])  # XOR output
    
    # Create a simple neural network with one hidden layer
    layers = [
        Layer(input_size=2, output_size=2),  # Hidden layer
        Layer(input_size=2, output_size=1)   # Output layer
    ]
    
    nn = NeuralNetwork(layers)
    nn.train(x_train, y_train, epochs=10000, learning_rate=0.1)

Epoch 1/10000, Loss: 0.2500034729052393
Epoch 2/10000, Loss: 0.25000346904911447
Epoch 3/10000, Loss: 0.250003465199395
Epoch 4/10000, Loss: 0.2500034613560666
Epoch 5/10000, Loss: 0.25000345751911535
Epoch 6/10000, Loss: 0.2500034536885271
Epoch 7/10000, Loss: 0.25000344986428785
Epoch 8/10000, Loss: 0.2500034460463836
Epoch 9/10000, Loss: 0.25000344223480014
Epoch 10/10000, Loss: 0.2500034384295238
Epoch 11/10000, Loss: 0.2500034346305408
Epoch 12/10000, Loss: 0.25000343083783705
Epoch 13/10000, Loss: 0.2500034270513989
Epoch 14/10000, Loss: 0.2500034232712124
Epoch 15/10000, Loss: 0.250003419497264
Epoch 16/10000, Loss: 0.25000341572954
Epoch 17/10000, Loss: 0.2500034119680266
Epoch 18/10000, Loss: 0.2500034082127104
Epoch 19/10000, Loss: 0.25000340446357755
Epoch 20/10000, Loss: 0.25000340072061467
Epoch 21/10000, Loss: 0.25000339698380825
Epoch 22/10000, Loss: 0.2500033932531448
Epoch 23/10000, Loss: 0.2500033895286109
Epoch 24/10000, Loss: 0.25000338581019305
Epoch 25/10000, Loss